In [36]:
# Load Data from interests file csv
import pandas as pd

categs = ["Sports",	"Music",	"Art and Creativity",	"Travel",	"Technology and Gaming"	,"Life Style",	"Food & Cooking",	"Volunteer"]
interests_data = pd.read_csv('sample_data/interests.csv',names = categs,header=None)

print(interests_data.loc[3]['Sports'])
#print(interests_data.info())



soccer


In [67]:
# For training such models we need synthetic data
#!pip install Faker
from faker import Faker
import random
fake = Faker()
fake.name()
print(dir(Faker))
# All we care about users
# What and how data are gonna be put inside csv? like for interests and stuff
# Coordinates belong to users insitutes
user_cols = ['int1','int2','int3','int4','int5','age','coordX','coordY']
insts_cols = ['uni','inst','x','y']

inst_data = pd.read_csv('sample_data/insts.csv',names = insts_cols,header = None)
inst_data
# Fake users and put them into csv
data = {'int1':'football','int2':'classical','int3':'photography','int4':'backpacking','int5':'video games','age':22,'coordX':'35.764433072675885','coordY':'10.84075666747697'}
df = pd.DataFrame([data])
for x in range(1000):
  # open interest folder and get random interests to put in the dictionary
  selected_interests = [
      random.choice(interests_data['Sports']),
      random.choice(interests_data['Music']),
      random.choice(interests_data['Art and Creativity']),
      random.choice(interests_data['Travel']),
      random.choice(interests_data['Technology and Gaming']),
      ]
  row = random.choice(range(len(inst_data)))

  new_row = pd.DataFrame({
                  'int1':[selected_interests[0]],
                  'int2':[selected_interests[1]],
                  'int3':[selected_interests[2]],
                  'int4':[selected_interests[3]],
                  'int5':[selected_interests[4]],
                  'age':[random.choice(range(19,25))],
                  'coordX':[inst_data.loc[row]['x']],
                  'coordY':[inst_data.loc[row]['y']]
                  })
  #print(row)
  df = pd.concat([df, new_row], ignore_index=True)
  #print(inst_data.loc[row])
  #print(inst_data.loc[row]['y'])

#print(inst_data.loc[1]['x'])
#print(inst_data.loc[1]['y'])
#print(df)

df.to_csv('sample_data/synth_users.csv', mode='w', header=False)

['__annotations__', '__class__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_map_provider_method', '_select_factory', '_select_factory_choice', '_select_factory_distribution', 'cache_pattern', 'factories', 'generator_attrs', 'items', 'locales', 'random', 'seed', 'seed_instance', 'seed_locale', 'unique', 'weights']


In [ ]:
# CollaborativeFiltering with KNN
# Version with Insitute Geolocation
# User data should include institutes Geo-coordinates within the DataFrame
# Facilitate the complexity ; as if the coordinates were the users coordinates no?
# If distance is 0 then users belong to the same Institute and of course both should be recommended to each others


import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic

class CollaborativeFilteringModel:
    def __init__(self, user_interests, user_locations, institutes, max_distance=2.0, K=2):
        self.user_interests = user_interests
        self.user_locations = user_locations
        self.institutes = institutes
        self.max_distance = max_distance
        self.K = K
        self.similarity_matrix = None
        self.knn_model = None
        self.build_model()

    def build_model(self):
        # Calculate cosine similarity matrix
        self.similarity_matrix = cosine_similarity(self.user_interests)

        # Initialize KNN model
        self.knn_model = NearestNeighbors(n_neighbors=self.K, metric='cosine')
        self.knn_model.fit(self.user_interests)

    def generate_recommendations(self, user_index):
        user = self.user_interests[user_index]
        _, indices = self.knn_model.kneighbors([user])

        recommendations = []
        user_location = self.user_locations[user_index]

        for idx in indices[0]:
            neighbor_user = self.user_interests[idx]
            neighbor_location = self.user_locations[idx]

            # Check geolocation proximity
            distance = geodesic(user_location, neighbor_location).km
            if distance <= self.max_distance:
                common_interests = np.logical_and(user, neighbor_user)
                recommended_users = np.where(common_interests)[0]
                recommendations.extend(recommended_users)

        return list(set(recommendations))  # Remove duplicates


In [ ]:
# Following Section is for exploiting the preceding code
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Read the user data from the CSV file
user_data = pd.read_csv('sample_data/synth_users.csv')

# Extract the necessary information from the user data
user_interests = user_data.iloc[:, 1:].values  # Assuming interests start from column index 1
user_locations = user_data[['Latitude', 'Longitude']].values
institute = user_data['Institute'].values

# Calculate the geolocation distance matrix
distance_matrix = np.zeros((len(user_locations), len(user_locations)))
for i, location1 in enumerate(user_locations):
    for j, location2 in enumerate(user_locations):
        distance_matrix[i, j] = geodesic(location1, location2).km

# Create an instance of the CollaborativeFilteringModel
model = CollaborativeFilteringModel(user_interests, user_locations, institute, max_distance=2.0, K=2)

# Example usage
target_user_index = 0
recommended_users = model.generate_recommendations(target_user_index)

print("Recommended Users for User", target_user_index)
print(recommended_users)


In [ ]:
# Saving model with Pickle
import pickle
pickle.dump(model, open('model.pkl', 'wb'))
'''
Testing purposes
pickled_model = pickle.load(open('model.pkl', 'rb'))
pickled_model.predict(X_test)


'''